# File Setup

In [3]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import netCDF4
import math
import sys
import os
import itertools
import scipy.stats

# Load Pacific SLR Data

In [ ]:
# Site ID 1 (index 0) from Greg Garner is Tarawa

## Time series, quantile slices

In [ ]:
# List all AISs used
AISs = [
    "k14",
    "dp16",
    "dp21",
    "k14v",
    "dp16v",
    "dp21v"    
]

AIS_scenarios = {}

# Loop over each AIS type
for AIS in AISs:
    # Create list of rcp scenarios to loop over
    rcps = [
        "26",
        "45",
        "85"
    ]

    # have one colour per rcp
    colours = [
        'r',
        'k',
        'b'
    ]

    # Create an empty dictionary to store the quantile dataframes in
    rcp_scenarios = {}

    # loop over each rcp
    for rcp,colour in zip(rcps,colours):

        # get file name 
        file = "pac_islands/workflows/wf_{}/rcp{}/total-workflow_localsl.nc".format(AIS,rcp,rcp)

        # Load the data
        ds = netCDF4.Dataset(file)

        # Extract the variables
        time = np.array(ds.variables['years'])
        lat = np.array(ds.variables['lat'])
        long = np.array(ds.variables['lon'])
        localSL_quantiles = np.array(ds.variables['localSL_quantiles'])
        quantiles = np.array(ds.variables['quantiles'])
        site_id = np.array(ds.variables['id'])

        # get some quantiles for plotting (indexing them from the ds quantile list)
        quantile_value = 0.5
        quantile_index = np.where(quantiles==round((np.min(abs(quantiles-quantile_value))+quantile_value),2))[0][0] ## check the rounding here
        quantile_index

        median_quantile = pd.DataFrame(data=localSL_quantiles[quantile_index,:,:],index=site_id,columns=time)
        median_quantile.index = median_quantile.index.set_names(['site'])
        median_quantile = pd.melt(median_quantile.reset_index(),id_vars='site')
        median_quantile.rename(columns={'value':'SLR','variable':'year'},inplace=True)

        # Lower IPCC likelihood quantile
        quantile_value = 0.17
        quantile_index = np.where(quantiles==round((np.min(abs(quantiles-quantile_value))+quantile_value),2))[0][0] ## check the rounding here
        quantile_index

        lower_quantile = pd.DataFrame(data=localSL_quantiles[quantile_index,:,:],index=site_id,columns=time)
        lower_quantile.index = lower_quantile.index.set_names(['site'])
        lower_quantile = pd.melt(lower_quantile.reset_index(),id_vars='site')
        lower_quantile.rename(columns={'value':'SLR','variable':'year'},inplace=True)

        # Upper IPCC likelihood quantile
        quantile_value = 0.83
        quantile_index = np.where(quantiles==round((np.min(abs(quantiles-quantile_value))+quantile_value),2))[0][0] ## check the rounding here
        quantile_index

        upper_quantile = pd.DataFrame(data=localSL_quantiles[quantile_index,:,:],index=site_id,columns=time)
        upper_quantile.index = upper_quantile.index.set_names(['site'])
        upper_quantile = pd.melt(upper_quantile.reset_index(),id_vars='site')
        upper_quantile.rename(columns={'value':'SLR','variable':'year'},inplace=True)

        # Get just one site for visualisations (since they're all very similar)
        median_quantile = median_quantile[median_quantile.site==0]
        lower_quantile = lower_quantile[lower_quantile.site==0]
        upper_quantile = upper_quantile[upper_quantile.site==0]

        rcp_scenarios.update({
            rcp:{
                "median":median_quantile,
                "lower":lower_quantile,
                "upper":upper_quantile,
                'colour':colour
            }
        })
        
    AIS_scenarios.update({
        AIS:rcp_scenarios
    })


# Visulise the projections

In [ ]:
# Set up the figure
fig = plt.figure(figsize=(15,10))
plt.subplots_adjust(hspace=0)

ax1 = plt.subplot2grid((3,2),(0,0))
ax2 = plt.subplot2grid((3,2),(1,0))
ax3 = plt.subplot2grid((3,2),(2,0))
ax4 = plt.subplot2grid((3,2),(0,1))
ax5 = plt.subplot2grid((3,2),(1,1))
ax6 = plt.subplot2grid((3,2),(2,1))

def plot_slr_scenarios(scenario_name,ax):
    '''
    '''
    for rcp,quantile_dict in AIS_scenarios[scenario_name].items():
        ax.plot(quantile_dict['median'].year,quantile_dict['median'].SLR,c=quantile_dict['colour'])
        ax.fill_between(x=list(quantile_dict['lower'].year),
                         y1=list(quantile_dict['lower'].SLR),
                         y2=list(quantile_dict['upper'].SLR),
                         zorder=-1000,
                         color=quantile_dict['colour'],
                         alpha=0.1)
    return(ax)

# plot the results
ax1 = plot_slr_scenarios('k14',ax1)
ax2 = plot_slr_scenarios('dp16',ax2)
ax3 = plot_slr_scenarios('dp21',ax3)
ax4 = plot_slr_scenarios('k14v',ax4)
ax5 = plot_slr_scenarios('dp16v',ax5)
ax6 = plot_slr_scenarios('dp21v',ax6)
  
# Format the graph
for ax in [ax2,ax5]:
    ax.set_ylabel('Mean sea-level change (mm)')

ax5.yaxis.set_label_position('right')
    
for ax in [ax3,ax6]:
    ax.set_xlabel('Time (years)')
# ax1.set_title()

for ax in [ax4,ax5,ax6]:
    ax.yaxis.tick_right()
    
for ax in [ax1,ax2,ax4,ax5]:
    ax.set_xticklabels([])
    
ax1.set_title('No VLM')
ax4.set_title('With VLM')

ax1.text(x=2020,y=np.max(ax1.get_ylim())-0.2*np.mean(ax1.get_ylim()),s='k14')
ax2.text(x=2020,y=np.max(ax2.get_ylim())-0.2*np.mean(ax2.get_ylim()),s='dp16')
ax3.text(x=2020,y=np.max(ax3.get_ylim())-0.2*np.mean(ax3.get_ylim()),s='dp21')

l1 = plt.scatter([],[],c='r')
l2 = plt.scatter([],[],c='k')
l3 = plt.scatter([],[],c='b')

ax6.legend([l1,l2,l3],['RCP 2.6','RCP 4.5','RCP 8.5'],loc='upper left')

plt.show()

In [ ]:
AIS_scenarios

In [ ]:
pd.concat(AIS_scenarios['k14v']['26'])

In [ ]:
AIS_scenarios['dp21']['85']['median']

In [ ]:
AIS_scenarios['dp21v']['85']['median']